In [ ]:
# Set Jupyter to render directly to the screen
%matplotlib inline

# Import pandas and numpy for analysis
import pandas as pd
import numpy as np

# Importing the Analysis Functions from the external Python Module

In [ ]:
# Import the functions to do the cointegration analysis. This requires that 'cointegration_analysis.py' is in the same 
# directory as this notebook.

from cointegration_analysis import estimate_long_run_short_run_relationships, engle_granger_two_step_cointegration_test

In [ ]:
# We can use the help() function in Python to bring up the docstrings. Alternatively, we could have used shift-tab while 
# highlighint the function name or looked in the cointegration_analysis.py file itself.

help(estimate_long_run_short_run_relationships)

In [ ]:
help(engle_granger_two_step_cointegration_test)

# Reading in the CSV file

In [ ]:
def read_data(filename):
    '''
    This function reads the .csv stored at the 'filename' location and returns a DataFrame
    with two levels of column names. The first level column contains the Stock Name and the 
    second contains the type of market data, e.g. bid/ask, price/volume.
    '''
    df = pd.read_csv(filename, index_col=0)
    df.columns = [df.columns.str[-2:], df.columns.str[:-3]]

    return df

In [ ]:
# Read the market data

filename = 'Pairs Trading.csv'
market_data = read_data(filename)

In [ ]:
# Get all the stock names into a list

stock_names = list(market_data.columns.get_level_values(0).unique())

print(stock_names)

# Some Examples

In [ ]:
# What is in the dataframe? (Display top 5 rows with the .head() DataFrame method.)

market_data.head()

In [ ]:
# Extracting the BidVolumes of AA. Note how the printed output is formatted differently 
# from the output printed for the whole DataFrame, to indicate that the single column 
# we extracted is a Series.

bid_volumes_AA = market_data['AA', 'BidVolume']
bid_volumes_AA.head()

In [ ]:
# Extracting the BidVolume at a specific time. This is just a number (float).

time = '2018-01-05 10:20:00'

bid_volume_AA_at_time = market_data.loc[time, ('AA', 'BidVolume')]
bid_volume_AA_at_time

In [ ]:
# Extracting a subset of observations, here the 1220th to the 1225th.

market_data.iloc[1220:1225]

In [ ]:
# Adding a new column based on a calculation of old columns
# NOTE: The new column gets added to the far right of the DataFrame

bid_ask_spread_AA = market_data['AA', 'AskPrice'] - market_data['AA', 'BidPrice']

market_data['AA', 'BidAskSpread'] = bid_ask_spread_AA

market_data.head()

In [ ]:
# The resulting DataFrame still looks a bit disordered, with the new AA column not alongside
# the other AA columns. We can alphabetically sort the column names to clean things up a bit.

market_data = market_data.sort_index(axis=1)
market_data.head()

In [ ]:
# If we want to iterate over each timestamp, we can easily do so. As practice, let's 
# calculate the maximum BidAskSpread of AA seen in the whole dataset.

max_spread_seen = 0
max_spread_seen_time = None

for time, mkt_data_at_time in market_data.iterrows():
    spread = mkt_data_at_time['AA', 'BidAskSpread'] 
    
    if spread > max_spread_seen:
        max_spread_seen = spread
        max_spread_seen_time = time

print(max_spread_seen_time, max_spread_seen)

In [ ]:
# Now let's do a comparison between different timestamps. We will calculate the maximum 
# price-increase of the AA BidPrice for the whole dataset.

max_price_increase = -999999
max_price_increase_seen_time = None

prev_time = None

for time, mkt_data_at_time in market_data.iterrows():
    if prev_time == None:
        # Skip the first observation, there is no previous bid price to compare to yet.
        prev_time = time
        continue
    
    previous_bid_price = market_data.loc[prev_time, ('AA', 'BidPrice')]  
    current_bid_price = mkt_data_at_time['AA', 'BidPrice']
       
    bid_price_increase = current_bid_price - previous_bid_price
    
    if bid_price_increase > max_price_increase:
        max_price_increase = bid_price_increase
        max_price_increase_seen_time = time
        
    # Update the previous time for the next iteration of the loop.
    prev_time = time

print(max_price_increase_seen_time, max_price_increase)

# Exercise

In [ ]:
# Now it's your turn. Start with plotting some of the values in the market_data DataFrame. Next, apply
# the analysis functions imported above. Finally design and backtest a trading algorithm exploting
# the cointegration relationships found in the data.